In [4]:
import os
import sys
import pandas as pd

from matplotlib import pyplot as plt

sys.path.append("../utils/")
from draw_court import draw_court
from shot_chart_plots import plot_scatter, joint_plot
%matplotlib

plt.close("all")

Using matplotlib backend: Qt5Agg


In [5]:
data_dir = "c:/Users/giase/Documents/DataScience/Projects/euroleague_api/notebooks/data"

In [6]:
file = "shot_data_2023.csv"

In [7]:
df = pd.read_csv(os.path.join(data_dir, file))

In [8]:
df["ZONE"].unique()

array(['I', 'E', 'C', 'H', 'D', ' ', 'F', 'B', 'A', 'G', 'J'],
      dtype=object)

In [9]:
# df.loc[df["ZONE"] == " ", "ACTION"].unique()

In [10]:
dd = df.pivot_table(index="TEAM", columns="ZONE", values="ID_ACTION", aggfunc="count")

In [11]:
mid_range_zones = ["D", "E", "F", "G"]

In [12]:
non_mid_range_zones = [u for u in df["ZONE"].unique() if u not in mid_range_zones + [" "]]

In [58]:
# df[df["TEAM"] == "PAM"]

In [14]:
(dd[mid_range_zones].sum(axis=1) / dd[non_mid_range_zones + mid_range_zones].sum(axis=1)).sort_values()

TEAM
OLY    0.128700
RED    0.130435
MAD    0.139334
BAS    0.154299
ZAL    0.155019
PAN    0.161290
ULK    0.176895
BER    0.179803
VIR    0.181034
IST    0.187735
MIL    0.189295
PAR    0.195282
MUN    0.204388
ASV    0.205596
PAM    0.230189
TEL    0.230947
BAR    0.233782
MCO    0.243667
dtype: float64

In [49]:
game_report_df = pd.read_csv(os.path.join(data_dir, "game_report_2023.csv"))

In [50]:
game_report_df.head()

,Season,Phase,Gamecode,Round,roundAlias,roundName,played,date,confirmedDate,confirmedHour,...,local.standingsScore,road.club.code,road.club.name,road.club.abbreviatedName,road.club.editorialName,road.club.tvCode,road.club.isVirtual,road.club.images.crest,road.score,road.standingsScore
0,2023,RS,1,1,Round 1,Round 1,True,2023-10-05T19:00:00,True,True,...,94,ASV,LDLC ASVEL Villeurbanne,ASVEL,ASVEL,ASV,False,https://media-cdn.incrowdsports.com/e33c6d1a-9...,73,73
1,2023,RS,2,1,Round 1,Round 1,True,2023-10-05T20:30:00,True,True,...,91,IST,Anadolu Efes Istanbul,Efes,Efes,EFS,False,https://media-cdn.incrowdsports.com/8ea8cec7-d...,74,74
2,2023,RS,3,1,Round 1,Round 1,True,2023-10-05T20:30:00,True,True,...,80,BER,ALBA Berlin,ALBA,ALBA,BER,False,https://media-cdn.incrowdsports.com/ccc34858-2...,68,68
3,2023,RS,4,1,Round 1,Round 1,True,2023-10-05T20:05:00,True,True,...,96,PAR,Partizan Mozzart Bet Belgrade,Partizan,Partizan,PAR,False,https://media-cdn.incrowdsports.com/2681304e-7...,81,81
4,2023,RS,5,1,Round 1,Round 1,True,2023-10-05T20:30:00,True,True,...,79,ZAL,Zalgiris Kaunas,Zalgiris,Zalgiris,ZAL,False,https://media-cdn.incrowdsports.com/0aa09358-3...,82,82


In [54]:
abb_to_name_dict = game_report_df[["road.club.code", "road.club.name"]].drop_duplicates().set_index("road.club.code").to_dict()["road.club.name"]

In [20]:
game_report_df = game_report_df[["Gamecode", "Round", "local.club.code", "road.club.code", "local.score", "road.score"]]

In [55]:
df["ID_ACTION"].unique()

array(['3FGA', '2FGM', '3FGM', 'FTM', '2FGA'], dtype=object)

In [56]:
df["TEAM"].unique()

array(['ASV', 'RED', 'BAR', 'IST', 'MUN', 'BER', 'PAR', 'TEL', 'ZAL',
       'VIR', 'ULK', 'MIL', 'OLY', 'PAN', 'BAS', 'MAD', 'PAM', 'MCO'],
      dtype=object)

In [57]:
df["Round"].unique()

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13], dtype=int64)

In [81]:
for zone in ["F"]:#df.ZONE.unique():
    tmp = df[df["ZONE"] == zone]
    made = tmp[tmp['ID_ACTION'].isin(['2FGM', '3FGM'])]
    missed = tmp[tmp['ID_ACTION'].isin(['2FGA', '3FGA'])]
    plot_scatter(made, missed, title=zone)

In [97]:
mid_range_zones = ["D", "E", "F", "G"]

In [98]:
midrange_df = df[df["ZONE"].isin(mid_range_zones) & df["ID_ACTION"].isin(['2FGM', '2FGA'])]

In [162]:
n_shots_df = df.groupby(["Gamecode", "TEAM"])[["Season"]].count()

In [163]:
n_midrange_df = midrange_df.groupby(["Gamecode", "TEAM"])[["Season"]].count()

In [178]:
merged_df = n_midrange_df.merge(n_shots_df, left_index=True, right_index=True, how="outer")
merged_df.columns = ["midrange", "total"]
merged_df.fillna(0, inplace=True)

In [179]:
# n_midrange_df[n_midrange_df.index.get_level_values(0) == 76]

In [180]:
# df[(df["Gamecode"] == 105) & (df["TEAM"] == "TEL") & (df["ID_ACTION"] == "3FGA") & df["ZONE"].isin(mid_range_zones)]

In [181]:
# midrange_df

In [184]:
merged_df

midrange  total
Gamecode TEAM                 
1        ASV       16.0     66
         RED       12.0     82
2        BAR       14.0     71
         IST       10.0     70
3        BER       10.0     69
...                 ...    ...
115      RED       11.0     87
116      TEL       22.0     90
         VIR       13.0     85
117      MCO       16.0     85
         ZAL        8.0     77

[234 rows x 2 columns]

In [195]:
n_made_missed_midrange_df = midrange_df.groupby(["Gamecode", "TEAM", "ID_ACTION"])[["Season"]].count()
n_made_missed_df = df[df["ID_ACTION"].isin(['2FGM', '2FGA'])].groupby(["Gamecode", "TEAM", "ID_ACTION"])[["Season"]].count()

In [198]:
merged2 = n_made_missed_midrange_df.merge(n_made_missed_df, left_index=True, right_index=True, how="outer")
merged2.columns = ["midrage", "total"]
merged2.fillna(0, inplace=True)

In [199]:
# n_made_missed_midrange_df.iloc[-61:-40]

In [203]:
n_shots_df = df[df["ID_ACTION"].isin(['2FGM', '3FGM', '2FGA', '3FGA'])].groupby(["Gamecode", "TEAM"])["Season"].count()

In [204]:
# n_midrange_df.shape

In [205]:
merged3_df = pd.merge(merged_df, n_shots_df, left_index=True, right_index=True)

In [206]:
merged3_df

Season_x  Season_y  Season
Gamecode TEAM                            
1        ASV         16        54      54
         RED         12        71      71
2        BAR         14        62      62
         IST         10        67      67
3        BER         10        54      54
...                 ...       ...     ...
115      RED         11        74      74
116      TEL         22        70      70
         VIR         13        65      65
117      MCO         16        59      59
         ZAL          8        73      73

[233 rows x 3 columns]

In [107]:
merged_df.columns = ["Mid-range", "Total"]

In [108]:
merged_df["perc"] = merged_df["Mid-range"] / merged_df["Total"]

In [109]:
merged_df

Mid-range  Total      perc
Gamecode TEAM                            
1        ASV          16     54  0.296296
         RED          12     71  0.169014
2        BAR          14     62  0.225806
         IST          10     67  0.149254
3        BER          10     54  0.185185
...                  ...    ...       ...
115      RED          11     74  0.148649
116      TEL          22     70  0.314286
         VIR          13     65  0.200000
117      MCO          16     59  0.271186
         ZAL           8     73  0.109589

[233 rows x 3 columns]

In [96]:
merged_df["t"] = ["A", "B"] * int(merged_df.shape[0] / 2 )
merged_reshaped_df = merged_df.reset_index().pivot(index="Gamecode", columns="t")[["TEAM", "perc"]]

ValueError: Length of values (232) does not match length of index (233)

In [63]:
merged_reshaped_df.columns = ["Team_A", "Team_B", "Midrange_perc_A", "Midrange_perc_B"]

In [67]:
# merged_reshaped_df.columns

In [65]:
game_report_df.head()

,Gamecode,Round,local.club.code,road.club.code,local.score,road.score
0,1,1,RED,ASV,94,73
1,2,1,BAR,IST,91,74
2,3,1,MUN,BER,80,68
3,4,1,TEL,PAR,96,81
4,5,1,VIR,ZAL,79,82


In [110]:
master_table = merged_reshaped_df.reset_index().merge(game_report_df, on="Gamecode")

In [111]:
master_table["home_win"] = master_table["local.score"] > master_table["road.score"]

In [112]:
master_table.head()

,Gamecode,Team_A,Team_B,Midrange_perc_A,Midrange_perc_B,Round,local.club.code,road.club.code,local.score,road.score,home_win
0,1,ASV,RED,0.296296,0.169014,1,RED,ASV,94,73,True
1,2,BAR,IST,0.225806,0.149254,1,BAR,IST,91,74,True
2,3,BER,MUN,0.185185,0.134328,1,MUN,BER,80,68,True
3,4,PAR,TEL,0.214286,0.159420,1,TEL,PAR,96,81,True
4,5,VIR,ZAL,0.171875,0.155172,1,VIR,ZAL,79,82,False


In [113]:
# master_table.loc[i, ["Team_A", "Team_B"]] = row[["Team_B", "Team_A"]].values

In [114]:
for i, row in master_table.iterrows():
    # break
    if (row[["Team_A", "Team_B"]].values == row[["local.club.code", "road.club.code"]].values).all():
        continue
    else:
        master_table.loc[i, ["Team_A", "Team_B"]] = row[["Team_B", "Team_A"]].values
        master_table.loc[i, ["Midrange_perc_A", "Midrange_perc_B"]] = row[["Midrange_perc_B", "Midrange_perc_A"]].values

In [116]:
master_table["home_midrange_volume"] = master_table["Midrange_perc_A"] > master_table["Midrange_perc_B"]

In [125]:
# n1 = ((master_table["Midrange_perc_A"] < master_table["Midrange_perc_B"]) & (master_table["local.score"] > master_table["road.score"])

In [126]:
master_table.pivot_table(index="home_midrange_volume", columns="home_win", values="Gamecode", aggfunc="count")

home_win,False,True
home_midrange_volume,,
False,2,7
True,8,6


In [127]:
13/23

0.5652173913043478

In [75]:
# master_table.apply(lambda row: row["Team_A", "TeamB"] == row["local.club.code", "road.club.code"])

In [45]:
# merged_df.reset_index().pivot(index="Gamecode", columns="TEAM", values="perc")

In [42]:
# merged_df.reset_index().pivot_table(index="Gamecode", columns="TEAM", values="perc")

In [76]:
merged_df

Mid-range  Total      perc  t
Gamecode TEAM                               
1        ASV          16     54  0.296296  A
         RED          12     71  0.169014  B
2        BAR          14     62  0.225806  A
         IST          10     67  0.149254  B
3        BER          10     54  0.185185  A
         MUN           9     67  0.134328  B
4        PAR          12     56  0.214286  A
         TEL          11     69  0.159420  B
5        VIR          11     64  0.171875  A
         ZAL           9     58  0.155172  B
6        MIL           9     62  0.145161  A
         ULK          13     65  0.200000  B
7        OLY           5     67  0.074627  A
         PAN          12     56  0.214286  B
8        BAS          17     78  0.217949  A
         MAD           8     68  0.117647  B
9        MCO          17     57  0.298246  A
         PAM          10     54  0.185185  B
10       BAS           6     68  0.088235  A
         BER          14     67  0.208955  B
11       IST          20     69  0.289855  A
         MAD           2     63  0.031746  B
13       MUN          11     71  0.154930  A
         PAN          11     52  0.211538  B
14       ASV           5     66  0.075758  A
         PAR           8     60  0.133333  B
15       MCO          12     64  0.187500  A
         VIR           9     61  0.147541  B
16       BAR           7     49  0.142857  A
         OLY          10     63  0.158730  B
17       PAM          20     62  0.322581  A
         ULK           5     65  0.076923  B
18       RED          11     69  0.159420  A
         ZAL           8     62  0.129032  B
19       ASV          13     62  0.209677  A
         IST           6     54  0.111111  B
20       MIL          13     51  0.254902  A
         OLY           8     53  0.150943  B
21       PAN           5     55  0.090909  A
         ULK           7     61  0.114754  B
22       BAR          15     71  0.211268  A
         PAR          14     62  0.225806  B
23       MAD          13     59  0.220339  A
         ZAL          18     72  0.250000  B
24       BAS          11     64  0.171875  A
         MUN          14     64  0.218750  B